In [34]:
from experiments._60_nlpmodels.fit_social_lda_models import *

# Comparación sobre tuning set

In [35]:
from bokeh.charts import Line, show, output_notebook
output_notebook()

Loading BokehJS ...

In [36]:
nlp_uids = [u for u, f in load_nlp_selected_users()]

In [37]:
with open('scores/t10_f1s_valid_svc.json') as f:
    f1s_combined = json.load(f)

f1s_social = json.load(open('../_1_one_user_learn_neighbours/scores/f1s_valid_svc.json'))
f1s_social = {u: f for u, f in f1s_social.items() if u in nlp_uids}

sorted_uids = [u for u, f in sorted(f1s_social.items(), key=lambda x:x[1])]
indices = range(len(sorted_uids))

f1s_lda = np.array([f1s_combined[sorted_uids[i]] for i in indices])
f1s_s = np.array([f1s_social[sorted_uids[i]] for i in indices])

# build a dataset where multiple columns measure the same thing
data = {
    "social": f1s_s,
    "social+lda10": f1s_lda
}

labels = 'social social+lda10'.split()

# create a line chart where each column of measures receives a unique color and dash style
line = Line(data, y=labels,
            dash=labels,
            color=labels,
            legend_sort_field = 'color',
            legend_sort_direction = 'descending',
            xlabel='usuarios (ordenados por f1 del modelo social en datos de evaluación)',
            ylabel='puntaje f1 sobre datos de evaluación', legend=True)

show(line)

KeyError: u'171971212'

In [ ]:
sum(f1s_lda > f1s_s)

In [ ]:
np.mean(f1s_lda)

In [ ]:
np.mean(f1s_s)

# Comparación en test set

In [ ]:
with open('scores/t10_f1s_test_svc.json') as f:
    f1s_combined = json.load(f)

f1s_social = json.load(open('../_1_one_user_learn_neighbours/scores/f1s_testv_svc.json'))
f1s_social = {u: f for u, f in f1s_social.items() if u in nlp_uids}

sorted_uids = [u for u, f in sorted(f1s_social.items(), key=lambda x:x[1])]
indices = range(len(sorted_uids))

f1s_lda = np.array([f1s_combined[sorted_uids[i]] for i in indices])
f1s_s = np.array([f1s_social[sorted_uids[i]] for i in indices])

# build a dataset where multiple columns measure the same thing
data = {
    "social": f1s_s,
    "social+lda10": f1s_lda
}

labels = 'social social+lda10'.split()

# create a line chart where each column of measures receives a unique color and dash style
line = Line(data, y=labels,
            dash=labels,
            color=labels,
            legend_sort_field = 'color',
            legend_sort_direction = 'descending',
            xlabel='usuarios (ordenados por f1 del modelo social en datos de evaluación)',
            ylabel='puntaje f1 sobre datos de evaluación', legend=True)

show(line)

# Comparación en training set

In [ ]:
with open('scores/t10_f1s_train_svc.json') as f:
    f1s_combined = json.load(f)

f1s_social = json.load(open('../_1_one_user_learn_neighbours/scores/f1s_train_svc.json'))
f1s_social = {u: f for u, f in f1s_social.items() if u in nlp_uids}

sorted_uids = [u for u, f in sorted(f1s_social.items(), key=lambda x:x[1])]
indices = range(len(sorted_uids))

f1s_lda = np.array([f1s_combined[sorted_uids[i]] for i in indices])
f1s_s = np.array([f1s_social[sorted_uids[i]] for i in indices])

# build a dataset where multiple columns measure the same thing
data = {
    "social": f1s_s,
    "social+lda10": f1s_lda
}

labels = 'social social+lda10'.split()

# create a line chart where each column of measures receives a unique color and dash style
line = Line(data, y=labels,
            dash=labels,
            color=labels,
            legend_sort_field = 'color',
            legend_sort_direction = 'descending',
            xlabel='usuarios (ordenados por f1 del modelo social en datos de evaluación)',
            ylabel='puntaje f1 sobre datos de evaluación', legend=True)

show(line)